In [72]:
import numpy as np
import pandas as pd
from textblob import TextBlob
import matplotlib.pyplot as plt
import plotly.graph_objects as go 
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")

In [73]:
trump = pd.read_csv("Trumpall2.csv")
biden = pd.read_csv("Bidenall2.csv")

In [74]:
trump.head()

,user,text
0,manny_rosen,@sanofi please tell us how many shares the Cr...
1,osi_abdul,"https://t.co/atM98CpqF7 Like, comment, RT #P..."
2,Patsyrw,Your AG Barr is as useless &amp; corrupt as y...
3,seyedebrahimi_m,Mr. Trump! Wake Up! Most of the comments bel...
4,James09254677,After 4 years you think you would have figure...


In [75]:
biden.head()

,user,text
0,MarkHodder3,@JoeBiden And we’ll find out who won in 2026...
1,K87327961G,@JoeBiden Your Democratic Nazi Party cannot be...
2,OldlaceA,@JoeBiden So did Lying Barr
3,penblogger,@JoeBiden It's clear you didnt compose this tw...
4,Aquarian0264,@JoeBiden I will vote in person thank you.


In [76]:
trump['text'][0]

' @sanofi please tell us how many shares the Criminal Trump Enterprise owns.'

Sentiment Analysis

Now, I will get started with sentiment analysis. I will do it by using the Textblob package in Python. Here I will use this package to perform simple text classification in either positive or negative on the basis of sentiment analysis:

1 Polarity refers to the sentiment or emotion expressed in the text, ranging from negative to positive.

2 Subjectivity,suggests that the text contains some subjects or opinions, measures the extent to which the text is subjective or opinionated rather than objective

In [77]:
TextBlob(trump['text'][0]).sentiment.subjectivity

0.525

In [78]:
trump.shape

(2788, 2)

In [79]:
def find_polarity(review):
    return TextBlob(review).sentiment.polarity

In [80]:
find_polarity('I love people with good behaviour')

0.6

In [81]:
trump['Sentiment Polarity'] = trump['text'].apply(find_polarity)

In [82]:
trump.head()

,user,text,Sentiment Polarity
0,manny_rosen,@sanofi please tell us how many shares the Cr...,0.05
1,osi_abdul,"https://t.co/atM98CpqF7 Like, comment, RT #P...",0.00
2,Patsyrw,Your AG Barr is as useless &amp; corrupt as y...,-0.50
3,seyedebrahimi_m,Mr. Trump! Wake Up! Most of the comments bel...,0.50
4,James09254677,After 4 years you think you would have figure...,0.00


In [83]:
biden['Sentiment Polarity'] = biden['text'].apply(find_polarity)

In [84]:
biden.head()

,user,text,Sentiment Polarity
0,MarkHodder3,@JoeBiden And we’ll find out who won in 2026...,0.00
1,K87327961G,@JoeBiden Your Democratic Nazi Party cannot be...,0.00
2,OldlaceA,@JoeBiden So did Lying Barr,0.00
3,penblogger,@JoeBiden It's clear you didnt compose this tw...,0.05
4,Aquarian0264,@JoeBiden I will vote in person thank you.,0.00


Sentiment Polarity on Both the candidates:

Now I will add a new attribute in both the datasets by the name of “Expression Label”:

1 polarity > 0 :: positive

2 polarity < 0 :: negative

3 Polarity == 0 :: neutral

In [85]:
trump['Expression Label'] = np.where(trump['Sentiment Polarity'] > 0, 'positive', 'negative')
trump['Expression Label'][trump['Sentiment Polarity']==0] = 'neutral'

In [86]:
trump['Expression Label'][1]

'neutral'

In [87]:
biden['Expression Label'] = np.where(biden['Sentiment Polarity'] > 0, 'positive', 'negative')
biden['Expression Label'][biden['Sentiment Polarity']==0] = 'neutral'

In [88]:
biden['Expression Label'][560]

'positive'

In [89]:
biden.head()

,user,text,Sentiment Polarity,Expression Label
0,MarkHodder3,@JoeBiden And we’ll find out who won in 2026...,0.00,neutral
1,K87327961G,@JoeBiden Your Democratic Nazi Party cannot be...,0.00,neutral
2,OldlaceA,@JoeBiden So did Lying Barr,0.00,neutral
3,penblogger,@JoeBiden It's clear you didnt compose this tw...,0.05,positive
4,Aquarian0264,@JoeBiden I will vote in person thank you.,0.00,neutral


In [90]:
trump.shape

(2788, 4)

Drop all the tweets with neutral polarity

In [91]:
n_trump = trump[trump['Sentiment Polarity'] == 0.0000]
print(n_trump.shape)

cond1 = trump['Sentiment Polarity'].isin(n_trump['Sentiment Polarity'])
trump.drop(trump[cond1].index, inplace=True)
print(trump.shape)

(1464, 4)
(1324, 4)


In [92]:
n_biden = biden[biden['Sentiment Polarity'] == 0.0000]
print(n_biden.shape)

cond2 = biden['Sentiment Polarity'].isin(n_biden['Sentiment Polarity'])
biden.drop(biden[cond2].index, inplace=True)
print(biden.shape)

(1509, 4)
(1031, 4)


balance both the datasets

In [93]:
# trump
np.random.seed(10)
remove_n = 324
drop_indices = np.random.choice(trump.index, remove_n, replace=False)

df_subset_trump = trump.drop(drop_indices)
print(df_subset_trump.shape)

(1000, 4)


In [94]:
# biden
np.random.seed(10)
remove_n = 31
drop_indices = np.random.choice(biden.index, remove_n, replace=False)

df_subset_biden = biden.drop(drop_indices)
print(df_subset_biden.shape)

(1000, 4)


predict the US Elections, by analyzing the number of positive and negative sentiments in both the accounts:

In [95]:
df_subset_trump.columns

Index(['user', 'text', 'Sentiment Polarity', 'Expression Label'], dtype='object')

In [96]:
print("Trump Review")
count_1 = df_subset_trump.groupby('Expression Label').count()
print(count_1)
negative_per1 = (count_1['Sentiment Polarity'][0]/1000)*10
positive_per1 = (count_1['Sentiment Polarity'][1]/1000)*10
print("\n")
print("Biden Review")
count_2 = df_subset_biden.groupby('Expression Label').count()
print(count_2)
negative_per2 = (count_2['Sentiment Polarity'][0]/1000)*10
positive_per2 = (count_2['Sentiment Polarity'][1]/1000)*10

Trump Review
                  user  text  Sentiment Polarity
Expression Label                                
negative           449   449                 449
positive           551   551                 551


Biden Review
                  user  text  Sentiment Polarity
Expression Label                                
negative           393   393                 393
positive           607   607                 607


In [97]:
print(f"Trumps  positive review {positive_per1}%")
print(f"Bidens  positive review {positive_per2}%")

Trumps  positive review 5.510000000000001%
Bidens  positive review 6.07%


In [98]:
Politicians = ['Donald Trump', 'Joe Biden']
lis_pos = [positive_per1, positive_per2]
lis_neg = [negative_per1, negative_per2]

fig = go.Figure(
    data=[
        go.Bar(name='Positive', x=Politicians, y=lis_pos),
        go.Bar(name='Negative', x=Politicians, y=lis_neg)
    ]
)

In [99]:
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()